In [2]:
import numpy as np

def vogel_approximation_method(supply, demand, costs):
    # Convert costs to float to handle np.inf assignments
    costs = costs.astype(float)
    transportation_plan = np.zeros_like(costs)
    total_cost = 0

    # Keep track of remaining supply and demand
    remaining_supply = supply.copy()
    remaining_demand = demand.copy()

    while np.sum(remaining_supply) > 0 and np.sum(remaining_demand) > 0:
        # Calculate row and column penalties
        row_penalties = []
        col_penalties = []

        # Calculate row penalties
        for i in range(costs.shape[0]):
            if remaining_supply[i] > 0:
                row_cost = costs[i, :]
                valid_cost = sorted([cost for cost in row_cost if cost != np.inf])
                if len(valid_cost) > 1:
                    row_penalties.append(valid_cost[1] - valid_cost[0])
                else:
                    row_penalties.append(0)
            else:
                row_penalties.append(-1)  # Mark as exhausted

        # Calculate column penalties
        for j in range(costs.shape[1]):
            if remaining_demand[j] > 0:
                col_cost = costs[:, j]
                valid_cost = sorted([cost for cost in col_cost if cost != np.inf])
                if len(valid_cost) > 1:
                    col_penalties.append(valid_cost[1] - valid_cost[0])
                else:
                    col_penalties.append(0)
            else:
                col_penalties.append(-1)  # Mark as exhausted

        # Determine whether to consider row or column penalty
        max_row_penalty = max([penalty for penalty in row_penalties if penalty >= 0], default=-1)
        max_col_penalty = max([penalty for penalty in col_penalties if penalty >= 0], default=-1)

        if max_row_penalty >= max_col_penalty:
            row_idx = row_penalties.index(max_row_penalty)
            col_idx = np.argmin(costs[row_idx, :])  # Choose the lowest cost in the row
        else:
            col_idx = col_penalties.index(max_col_penalty)
            row_idx = np.argmin(costs[:, col_idx])  # Choose the lowest cost in the column

        # Allocate the supply and demand
        allocated = min(remaining_supply[row_idx], remaining_demand[col_idx])
        transportation_plan[row_idx, col_idx] = allocated
        total_cost += allocated * costs[row_idx, col_idx]

        # Update supply and demand
        remaining_supply[row_idx] -= allocated
        remaining_demand[col_idx] -= allocated

        # Mark exhausted rows and columns as np.inf
        if remaining_supply[row_idx] == 0:
            costs[row_idx, :] = np.inf
        if remaining_demand[col_idx] == 0:
            costs[:, col_idx] = np.inf

    return transportation_plan, total_cost

# Example usage
supply = np.array([300, 400, 500])
demand = np.array([250, 350, 400, 200])
costs = np.array([
    [3, 1, 7, 4],
    [2, 6, 5, 9],
    [8, 3, 3, 2]
])

plan, cost = vogel_approximation_method(supply, demand, costs)
print("Optimal Transportation Plan:")
print(plan)
print("Total Transportation Cost:", cost)

Optimal Transportation Plan:
[[  0. 300.   0.   0.]
 [250.   0. 150.   0.]
 [  0.  50. 250. 200.]]
Total Transportation Cost: 2850.0


In [3]:
import numpy as np
from scipy.optimize import linear_sum_assignment

def assignment_problem():
    print("\nAssignment Problem Solver (Hungarian Algorithm)")
    print("---------------------------------------------")

    n = int(input("Enter number of workers/jobs: "))

    print("\nEnter cost matrix (row = workers, column = jobs):")
    cost_matrix = []
    for i in range(n):
        row = list(map(float, input(f"Costs for worker {i+1} (space-separated): ").split()))
        cost_matrix.append(row)
    cost_matrix = np.array(cost_matrix)

    # Solve using Hungarian Algorithm
    row_ind, col_ind = linear_sum_assignment(cost_matrix)
    total_cost = cost_matrix[row_ind, col_ind].sum()

    print("\nOptimal Assignments:")
    for worker, job in zip(row_ind, col_ind):
        print(f"Worker {worker+1} → Job {job+1} (Cost: {cost_matrix[worker][job]:.2f})")
    print(f"Total Minimum Cost: {total_cost:.2f}")

assignment_problem()


Assignment Problem Solver (Hungarian Algorithm)
---------------------------------------------
Enter number of workers/jobs: 3

Enter cost matrix (row = workers, column = jobs):
Costs for worker 1 (space-separated): 1 2 3
Costs for worker 2 (space-separated): 3 2 1
Costs for worker 3 (space-separated): 2 5 6

Optimal Assignments:
Worker 1 → Job 2 (Cost: 2.00)
Worker 2 → Job 3 (Cost: 1.00)
Worker 3 → Job 1 (Cost: 2.00)
Total Minimum Cost: 5.00


In [6]:
import math

def eoq():
    print("\nEconomic Order Quantity (EOQ) Calculator")
    print("---------------------------------------")

    A = float(input("Enter annual demand (units): "))
    B = float(input("Enter ordering cost per order ($): "))
    C = float(input("Enter holding cost per unit/year ($): "))

    eoq = math.sqrt((2 * A * B) / C)
    orders_per_year = A / eoq
    total_cost = (A * B / eoq) + (C * eoq / 2)

    print("\nResults:")
    print(f"Optimal Order Quantity (EOQ): {eoq:.2f} units")
    print(f"Number of Orders/Year: {orders_per_year:.2f}")
    print(f"Total Annual Inventory Cost: ${total_cost:.2f}")

eoq()


Economic Order Quantity (EOQ) Calculator
---------------------------------------
Enter annual demand (units): 1000
Enter ordering cost per order ($): 5
Enter holding cost per unit/year ($): 2

Results:
Optimal Order Quantity (EOQ): 70.71 units
Number of Orders/Year: 14.14
Total Annual Inventory Cost: $141.42


In [8]:
def knapsack(weights, values, capacity):
    n = len(weights)  # number of items
    # Create a DP table with (n+1) rows and (capacity+1) columns
    dp = [[0] * (capacity + 1) for _ in range(n + 1)]

    # Build the DP table
    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if weights[i - 1] <= w:  # if the item can be included
                # Max of not including or including the item
                dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weights[i - 1]] + values[i - 1])
            else:
                dp[i][w] = dp[i - 1][w]

    # The bottom-right cell contains the maximum value we can get
    return dp[n][capacity]

# Example usage
weights = [2, 3, 4, 5]  # Weights of the items
values = [3, 4, 5, 6]   # Values of the items
capacity = 5            # Capacity of the knapsack

max_value = knapsack(weights, values, capacity)
print(f"Maximum value that can be obtained: {max_value}")

Maximum value that can be obtained: 7


In [9]:
from scipy.optimize import linprog

# Coefficients of the objective function (maximize revenue)
c = [-5, -3]  # negative because linprog minimizes by default

# Coefficients of the inequality constraints (LHS)
A = [
    [2, 1],  # 2x + y <= 500
    [1, 1]   # x + y <= 400
]

# RHS of the inequality constraints
b = [500, 400]

# Coefficients of the lower bounds (demand constraints)
x_bounds = (100, None)  # x >= 100
y_bounds = (50, None)   # y >= 50

# Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Output the result
print("Optimal number of chocolate cakes (x):", result.x[0])
print("Optimal number of vanilla cakes (y):", result.x[1])
print("Maximum revenue:", -result.fun)

Optimal number of chocolate cakes (x): 100.0
Optimal number of vanilla cakes (y): 300.0
Maximum revenue: 1400.0


In [10]:
import numpy as np

def simplex(c, A, b):

    num_vars = len(c)
    num_constraints = len(b)

    tableau = np.zeros((num_constraints + 1, num_vars + num_constraints + 1))

    for i in range(num_constraints):
        tableau[i, :num_vars] = A[i]
        tableau[i, num_vars + i] = 1
        tableau[i, -1] = b[i]

    tableau[-1, :num_vars] = -c

    while np.any(tableau[-1, :-1] < 0):

        pivot_col = np.argmin(tableau[-1, :-1])

        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios[ratios < 0] = np.inf
        pivot_row = np.argmin(ratios)

        pivot_value = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_value

        for i in range(num_constraints + 1):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

    optimal_solution = tableau[:-1, -1]
    max_value = tableau[-1, -1]

    return optimal_solution, max_value

c = np.array([2, 3, 1])

A = np.array([
    [1, 1, 1],
    [1, 2, -1]
])

b = np.array([4, 2])

optimal_solution, max_value = simplex(c, A, b)

print("Optimal solution (u1, u2, u3):", optimal_solution)
print("Maximum profit:", max_value)

Optimal solution (u1, u2, u3): [2. 2.]
Maximum profit: 8.0


In [11]:
def totalCost(mask, curr, n, cost, memo, path_tracker):
    if mask == (1 << n) - 1:        # if all cities are visited, return the cost to return to the starting city (0)
        return cost[curr][0]

    if memo[curr][mask] != -1:
        return memo[curr][mask]

    ans = float('inf')
    best_next_city = -1
    for i in range(n):      # visiting every city that has not been visited yet
        if (mask & (1 << i)) == 0:

          # If city i is not visited
          # Visit city i and update the mask
            result = cost[curr][i] + totalCost(mask | (1 << i), i, n, cost, memo, path_tracker)
            if result < ans:
                ans = result
                best_next_city = i

    # Memoize the result
    memo[curr][mask] = ans
    # Track the best path
    path_tracker[curr][mask] = best_next_city
    return ans


def reconstructPath(n, cost, memo, path_tracker):
    mask = 1  # start with city 0 visited
    curr = 0
    path = [curr]
    while True:
        next_city = path_tracker[curr][mask]
        if next_city == -1:
            break
        path.append(next_city)
        mask |= (1 << next_city)
        curr = next_city
    return path


def tsp(cost):
    n = len(cost)

    # Initialize memoization table with -1
    # (indicating uncomputed states)
    memo = [[-1] * (1 << n) for _ in range(n)]
    # Initialize path tracker to store the best path at each step
    path_tracker = [[-1] * (1 << n) for _ in range(n)]

    # Start from city 0, with only city 0 visited initially (mask = 1)
    total_cost = totalCost(1, 0, n, cost, memo, path_tracker)
    # Reconstruct the optimal path
    path = reconstructPath(n, cost, memo, path_tracker)
    return total_cost, path


# Example cost matrix
cost = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

# Call tsp function to get the minimum cost and the optimal path
total_cost, path = tsp(cost)

print(f"Minimum Cost: {total_cost}")
print(f"Optimal Path: {path}")

Minimum Cost: 80
Optimal Path: [0, 1, 3, 2]


In [12]:
from scipy.optimize import linprog

# Coefficients of the objective function (profit)
c = [-200, -150]  # Negating to convert maximization to minimization

# Coefficients of the inequality constraints (Ax <= b)
A = [
    [1, 1],         # x + y <= 60 (Total land constraint)
    [20, 10],       # 20x + 10y <= 1200 (Fertilizer constraint)
    [10, 15],       # 10x + 15y <= 600 (Insecticide constraint)
]

# Right-hand side of the constraints
b = [60, 1200, 600]

# Bounds for x and y (Non-negativity and minimum acres constraints)
x_bounds = (20, None)  # At least 20 acres of wheat
y_bounds = (10, None)  # At least 10 acres of barley

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Output the results
if result.success:
    x_opt = result.x[0]  # Optimal number of acres of wheat
    y_opt = result.x[1]  # Optimal number of acres of barley
    max_profit = -result.fun  # Convert back to the actual profit (negated in the objective)

    print(f"Optimal number of acres of wheat: {x_opt:.2f}")
    print(f"Optimal number of acres of barley: {y_opt:.2f}")
    print(f"Maximum profit: ${max_profit:.2f}")
else:
    print("No solution found.")

Optimal number of acres of wheat: 45.00
Optimal number of acres of barley: 10.00
Maximum profit: $10500.00


In [13]:
from scipy.optimize import linprog

# Coefficients of the objective function (profit)
c = [-20, -30]  # Negating to convert maximization to minimization

# Coefficients of the inequality constraints (Ax <= b)
A = [
    [1, 5],  # x + 5y <= 125 (Wood constraint)
    [3, 1],  # 3x + y <= 80 (Metal constraint)
]

# Right-hand side of the constraints
b = [125, 80]

# Bounds for x and y (non-negativity constraint)
x_bounds = (0, None)
y_bounds = (0, None)

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='highs')

# Output the results
if result.success:
    x_opt = result.x[0]  # Optimal number of chairs
    y_opt = result.x[1]  # Optimal number of tables
    max_profit = -result.fun  # Convert back to the actual profit (negated in the objective)

    print(f"Optimal number of chairs: {int(x_opt)}")
    print(f"Optimal number of tables: {int(y_opt)}")
    print(f"Maximum profit: ${max_profit:.2f}")
else:
    print("No solution found.")

Optimal number of chairs: 19
Optimal number of tables: 21
Maximum profit: $1025.00
